In [1]:
import pandas as pd
from math import comb
from ax.service.ax_client import AxClient
import time
import sys

sys.path.append('../../../0_helper_functions')
import sdlnano as sdl
import calculation as calc

In [2]:
# Design space size calculation


def calculate_combinations(num_variables, total_sum, step):
    steps = int(total_sum / step)
    return comb(steps + num_variables - 1, num_variables - 1)

# Define parameters
step = 0.05
total_sum = 1

# Calculate combinations for each group
count_group1 = calculate_combinations(10, total_sum, step)  # 3 Solid lipids + 3 Polymers + 3 Liquid lipids + 1 Drug
count_group2 = calculate_combinations(4, total_sum, step)   # 3 Surfactants + 1 Water

# Calculate total combinations
total_combinations = count_group1 * count_group2


print("organic design space size: ", round(count_group1/(1e6),2), 'Million')
print("aqueous design space size: ", round(count_group2/(1e3),2), 'Thousand')
print("design space size: ", round(total_combinations/(1e9),2), 'Billion')


organic design space size:  10.02 Million
aqueous design space size:  1.77 Thousand
design space size:  17.74 Billion


# Check the iteration number

In [3]:
iteration = sdl.get_iteration_number()
print("This is the iteration: #", iteration)

This is the iteration: # 6


# Check the drug

In [4]:
drug = 'ACE'
print("This is the drug:", drug)

This is the drug: ACE


# Optimizer initialization (N/A)

# Generate recommendations

In [5]:
time_start = time.time()

previous_optimizer_file_name = '../iteration_' + str(iteration-1) + '/optimizer/optimizer_load_' + str(iteration-1) + '.json'
# generate recomms
ax_client_init = AxClient.load_from_json_file(previous_optimizer_file_name)
unlabeled_trial, ax_client_design = sdl.generate_trials(ax_client=ax_client_init, num_of_trials = 16, drug = drug, bopt=1)

time_end = time.time()

print(round((time_end-time_start)/60), 'min')

/Users/zeqing/opt/anaconda3/envs/sdlnano_plot/lib/python3.11/site-packages/botorch/acquisition/multi_objective/logei.py:418: RuntimeWarning: `cache_root` is only supported for GPyTorchModels that are not MultiTask models and don't produce a TransformedPosterior. Got a model of type <class 'botorch.models.model.ModelList'>. Setting `cache_root = False`.
  NoisyExpectedHypervolumeMixin.__init__(
[INFO 08-06 15:20:47] ax.service.ax_client: Generated new trial 96 with parameters {'Drug': 90, 'SL_1': 71, 'SL_2': 0, 'SL_3': 0, 'LL_1': 0, 'LL_2': 0, 'LL_3': 0, 'P_1': 0, 'P_2': 0, 'P_3': 0, 'S_1': 0, 'S_2': 0, 'S_3': 2, 'Water': 0} using model BoTorch.
/Users/zeqing/opt/anaconda3/envs/sdlnano_plot/lib/python3.11/site-packages/ax/core/data.py:293: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the rel

11 min


In [6]:
unlabeled_trial[['S_1', 'S_2', 'S_3', 'Water']]

,S_1,S_2,S_3,Water
0,0,0,2,0
1,0,0,0,0
2,0,0,2,1
3,0,0,0,0
4,0,0,2,0
5,0,0,0,0
6,0,0,0,0
7,0,0,99,0
8,0,0,0,0
9,0,0,0,0


In [7]:
mask = (unlabeled_trial[['S_1', 'S_2', 'S_3', 'Water']] == 0).all(axis=1)

# print the rows that are about to change
print("Rows to be updated (before):")
print(unlabeled_trial.loc[mask])

# set Water to 100 on those rows
unlabeled_trial.loc[mask, 'Water'] = 100

# print the rows after the update
print("\nRows after update:")
print(unlabeled_trial.loc[mask])

Rows to be updated (before):
    trial_index  Drug  SL_1  SL_2  SL_3  LL_1  LL_2  LL_3  P_1  P_2  ...  S_1  \
1            97    85    34     0     0     0     0     0    0    0  ...    0   
3            99    85    34     0     0     0     0     0    0    0  ...    0   
5           101    85    34     0     0     0     0     0    0    0  ...    0   
6           102    85    34     0     0     0     0     0    0    0  ...    0   
8           104    85    34     0     0     0     0     0    0    0  ...    0   
9           105    85    34     0     0     0     0     0    0    0  ...    0   
11          107    99    72     0     0     0     0     0    0    0  ...    0   
12          108    99    72     0     0     0     0     0    0    0  ...    0   
14          110    85    34     0     0     0     0     0    0    0  ...    0   

    S_2  S_3  Water  Solu Solu_STD  Size Size_STD   PDI PDI_STD  
1     0    0      0  None     None  None     None  None    None  
3     0    0      0  None   

In [8]:
unlabeled_trial[['S_1', 'S_2', 'S_3', 'Water']]

,S_1,S_2,S_3,Water
0,0,0,2,0
1,0,0,0,100
2,0,0,2,1
3,0,0,0,100
4,0,0,2,0
5,0,0,0,100
6,0,0,0,100
7,0,0,99,0
8,0,0,0,100
9,0,0,0,100


In [9]:
# process the trails into ratios

unlabeled_trial_processed = sdl.process_trails(unlabeled_trial)
unlabeled_trial_processed.describe()

,trial_index,Drug,SL_1,SL_2,SL_3,LL_1,LL_2,LL_3,P_1,P_2,P_3,S_1,S_2,S_3,Water
count,16.000000,16.000000,16.000000,16.0,16.0,16.0,16.0,16.0,16.0,16.000000,16.0,16.0000,16.0000,16.000000,16.000000
mean,103.500000,0.660438,0.339187,0.0,0.0,0.0,0.0,0.0,0.0,0.000375,0.0,0.0625,0.0625,0.229187,0.645813
std,4.760952,0.075393,0.074852,0.0,0.0,0.0,0.0,0.0,0.0,0.001500,0.0,0.2500,0.2500,0.416690,0.478728
min,96.000000,0.556000,0.260000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0000,0.0000,0.000000,0.000000
25%,99.750000,0.576500,0.286000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0000,0.0000,0.000000,0.000000
50%,103.500000,0.714000,0.286000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0000,0.0000,0.000000,1.000000
75%,107.250000,0.714000,0.423500,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0000,0.0000,0.166750,1.000000
max,111.000000,0.740000,0.441000,0.0,0.0,0.0,0.0,0.0,0.0,0.006000,0.0,1.0000,1.0000,1.000000,1.000000


In [10]:
unlabeled_trial
unlabeled_trial.describe()

,trial_index,Drug,SL_1,SL_2,SL_3,LL_1,LL_2,LL_3,P_1,P_2,P_3,S_1,S_2,S_3,Water
count,16.000000,16.000000,16.000000,16.0,16.0,16.0,16.0,16.0,16.0,16.0000,16.0,16.0000,16.0000,16.000000,16.000000
mean,103.500000,89.062500,48.062500,0.0,0.0,0.0,0.0,0.0,0.0,0.0625,0.0,0.0625,0.0625,6.562500,56.375000
std,4.760952,5.458556,18.752666,0.0,0.0,0.0,0.0,0.0,0.0,0.2500,0.0,0.2500,0.2500,24.662978,51.089301
min,96.000000,85.000000,34.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0000,0.0000,0.000000,0.000000
25%,99.750000,85.000000,34.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0000,0.0000,0.000000,0.000000
50%,103.500000,85.000000,34.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0000,0.0000,0.000000,100.000000
75%,107.250000,91.250000,71.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0000,0.0000,0.500000,100.000000
max,111.000000,99.000000,72.000000,0.0,0.0,0.0,0.0,0.0,0.0,1.0000,0.0,1.0000,1.0000,99.000000,100.000000


In [11]:
# Convert ratios to volumes

transfer = calc.converter(unlabeled_trial_processed)
transfer.describe()

,trial_index,Drug,SL_1,SL_2,SL_3,LL_1,LL_2,LL_3,P_1,P_2,P_3,S_1,S_2,S_3,Water,Solvent
count,16.000000,16.000000,16.000000,16.0,16.0,16.0,16.0,16.0,16.0,16.00000,16.0,16.0,16.0,16.000000,16.000000,16.0
mean,103.500000,231.153125,118.715625,0.0,0.0,0.0,0.0,0.0,0.0,0.13125,0.0,62.5,62.5,229.187500,645.812500,0.0
std,4.760952,26.387607,26.198217,0.0,0.0,0.0,0.0,0.0,0.0,0.52500,0.0,250.0,250.0,416.690008,478.728067,0.0
min,96.000000,194.600000,91.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.000000,0.000000,0.0
25%,99.750000,201.775000,100.100000,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.000000,0.000000,0.0
50%,103.500000,249.900000,100.100000,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.000000,1000.000000,0.0
75%,107.250000,249.900000,148.225000,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,166.750000,1000.000000,0.0
max,111.000000,259.000000,154.350000,0.0,0.0,0.0,0.0,0.0,0.0,2.10000,0.0,1000.0,1000.0,1000.000000,1000.000000,0.0


In [12]:
#dead_volume = 3.0 # mL

round(transfer[['Drug','SL_1', 'SL_2', 'SL_3', 'LL_1', 'LL_2', 'LL_3', 'P_1', 'P_2', 'P_3', 'S_1', 'S_2', 'S_3', 'Water', 'Solvent']].sum()/1000,1)

Drug        3.7
SL_1        1.9
SL_2        0.0
SL_3        0.0
LL_1        0.0
LL_2        0.0
LL_3        0.0
P_1         0.0
P_2         0.0
P_3         0.0
S_1         1.0
S_2         1.0
S_3         3.7
Water      10.3
Solvent     0.0
dtype: float64

In [13]:
# file path
ax_client_design_path = "optimizer/optimizer_design_" + str(iteration) + ".json"

unlabeled_trial_path = "data/unlabeled_" + str(iteration) + ".xlsx"

unlabeled_trial_processed_path = "data/unlabeled_processed_" + str(iteration) + ".xlsx"

transfer_path = 'data/transfer_' + str(iteration) + '.xlsx'


In [14]:
ax_client_design.save_to_json_file(ax_client_design_path)

unlabeled_trial.to_excel(unlabeled_trial_path, index=False)

unlabeled_trial_processed.to_excel(unlabeled_trial_processed_path, index=False)

transfer.to_excel(transfer_path, index = False)

# Generate new protocol

In [15]:
# Paths
template_protocol = '../../templates/OT_protocol_template.py'

output_script_path_1 = 'protocol/iteration_' + str(iteration) + '_OT_2_protocol_1.py'
output_script_path_2 = 'protocol/iteration_' + str(iteration) + '_OT_2_protocol_2.py'



In [16]:
# Update the script with Excel data
updated_script = calc.update_transfer_script(template_protocol, transfer_path, output_script_path_1, output_script_path_2) 


In [17]:
updated_script

('protocol/iteration_6_OT_2_protocol_1.py',
 'protocol/iteration_6_OT_2_protocol_2.py')

# Perform experiment

# Results

In [18]:
results = calc.parameters_and_complexity(iteration)
results

,trial_index,Drug,SL_1,SL_2,SL_3,LL_1,LL_2,LL_3,P_1,P_2,P_3,S_1,S_2,S_3,Water,Complexity,Complexity_STD
0,96,0.559,0.441,0,0,0,0,0,0,0.000,0,0,0,1.000,0.000,2,0
1,97,0.714,0.286,0,0,0,0,0,0,0.000,0,0,0,0.000,1.000,1,0
2,98,0.559,0.441,0,0,0,0,0,0,0.000,0,0,0,0.667,0.333,2,0
3,99,0.714,0.286,0,0,0,0,0,0,0.000,0,0,0,0.000,1.000,1,0
4,100,0.556,0.438,0,0,0,0,0,0,0.006,0,0,0,1.000,0.000,3,0
5,101,0.714,0.286,0,0,0,0,0,0,0.000,0,0,0,0.000,1.000,1,0
6,102,0.714,0.286,0,0,0,0,0,0,0.000,0,0,0,0.000,1.000,1,0
7,103,0.740,0.260,0,0,0,0,0,0,0.000,0,0,0,1.000,0.000,2,0
8,104,0.714,0.286,0,0,0,0,0,0,0.000,0,0,0,0.000,1.000,1,0
9,105,0.714,0.286,0,0,0,0,0,0,0.000,0,0,0,0.000,1.000,1,0


In [19]:
size_raw = calc.size_raw(iteration)
size_raw

,Data Quality,Item,Size,PD Index
0,Good,A1,229.861660,0.148526
1,Good,A2,240.280546,0.231487
2,Good,A3,239.335405,0.193945
3,Good,A4,219.001469,0.176638
4,Caution,A5,222.040506,0.219892
5,Caution,A6,224.735623,0.209022
6,Good,B1,228.887888,0.296847
7,Good,B2,231.315533,0.281090
8,Good,B3,229.981428,0.382360
9,Caution,B4,213.594524,0.206105


In [20]:
size_processed = calc.process_formulations(size_raw)
size_processed

,Formulation,Size,Size_STD,PDI,PDI_STD,Formulation Quality
0,Formulation 1,236.492537,5.761920,0.191319,0.041543,1
1,Formulation 2,1000.000000,0.000000,1.000000,0.000000,0
2,Formulation 3,230.061616,1.215807,0.320099,0.054492,1
3,Formulation 4,1000.000000,0.000000,1.000000,0.000000,0
4,Formulation 5,233.580306,1.771304,0.282933,0.019797,1
5,Formulation 6,1000.000000,0.000000,1.000000,0.000000,0
6,Formulation 7,1000.000000,0.000000,1.000000,0.000000,0
7,Formulation 8,1000.000000,0.000000,1.000000,0.000000,0
8,Formulation 9,1000.000000,0.000000,1.000000,0.000000,0
9,Formulation 10,1000.000000,0.000000,1.000000,0.000000,0


In [21]:
results_final = pd.concat([results, size_processed], axis=1)
results_final

,trial_index,Drug,SL_1,SL_2,SL_3,LL_1,LL_2,LL_3,P_1,P_2,...,S_3,Water,Complexity,Complexity_STD,Formulation,Size,Size_STD,PDI,PDI_STD,Formulation Quality
0,96,0.559,0.441,0,0,0,0,0,0,0.000,...,1.000,0.000,2,0,Formulation 1,236.492537,5.761920,0.191319,0.041543,1
1,97,0.714,0.286,0,0,0,0,0,0,0.000,...,0.000,1.000,1,0,Formulation 2,1000.000000,0.000000,1.000000,0.000000,0
2,98,0.559,0.441,0,0,0,0,0,0,0.000,...,0.667,0.333,2,0,Formulation 3,230.061616,1.215807,0.320099,0.054492,1
3,99,0.714,0.286,0,0,0,0,0,0,0.000,...,0.000,1.000,1,0,Formulation 4,1000.000000,0.000000,1.000000,0.000000,0
4,100,0.556,0.438,0,0,0,0,0,0,0.006,...,1.000,0.000,3,0,Formulation 5,233.580306,1.771304,0.282933,0.019797,1
5,101,0.714,0.286,0,0,0,0,0,0,0.000,...,0.000,1.000,1,0,Formulation 6,1000.000000,0.000000,1.000000,0.000000,0
6,102,0.714,0.286,0,0,0,0,0,0,0.000,...,0.000,1.000,1,0,Formulation 7,1000.000000,0.000000,1.000000,0.000000,0
7,103,0.740,0.260,0,0,0,0,0,0,0.000,...,1.000,0.000,2,0,Formulation 8,1000.000000,0.000000,1.000000,0.000000,0
8,104,0.714,0.286,0,0,0,0,0,0,0.000,...,0.000,1.000,1,0,Formulation 9,1000.000000,0.000000,1.000000,0.000000,0
9,105,0.714,0.286,0,0,0,0,0,0,0.000,...,0.000,1.000,1,0,Formulation 10,1000.000000,0.000000,1.000000,0.000000,0


In [22]:
# µg/mL
results_final['Solu'] = results_final['Drug'] * 600 / 0.3
results_final['Solu_STD'] = 0

results_final.loc[results_final['Formulation Quality'] == 0, 'Solu'] = 0
results_final

,trial_index,Drug,SL_1,SL_2,SL_3,LL_1,LL_2,LL_3,P_1,P_2,...,Complexity,Complexity_STD,Formulation,Size,Size_STD,PDI,PDI_STD,Formulation Quality,Solu,Solu_STD
0,96,0.559,0.441,0,0,0,0,0,0,0.000,...,2,0,Formulation 1,236.492537,5.761920,0.191319,0.041543,1,1118.0,0
1,97,0.714,0.286,0,0,0,0,0,0,0.000,...,1,0,Formulation 2,1000.000000,0.000000,1.000000,0.000000,0,0.0,0
2,98,0.559,0.441,0,0,0,0,0,0,0.000,...,2,0,Formulation 3,230.061616,1.215807,0.320099,0.054492,1,1118.0,0
3,99,0.714,0.286,0,0,0,0,0,0,0.000,...,1,0,Formulation 4,1000.000000,0.000000,1.000000,0.000000,0,0.0,0
4,100,0.556,0.438,0,0,0,0,0,0,0.006,...,3,0,Formulation 5,233.580306,1.771304,0.282933,0.019797,1,1112.0,0
5,101,0.714,0.286,0,0,0,0,0,0,0.000,...,1,0,Formulation 6,1000.000000,0.000000,1.000000,0.000000,0,0.0,0
6,102,0.714,0.286,0,0,0,0,0,0,0.000,...,1,0,Formulation 7,1000.000000,0.000000,1.000000,0.000000,0,0.0,0
7,103,0.740,0.260,0,0,0,0,0,0,0.000,...,2,0,Formulation 8,1000.000000,0.000000,1.000000,0.000000,0,0.0,0
8,104,0.714,0.286,0,0,0,0,0,0,0.000,...,1,0,Formulation 9,1000.000000,0.000000,1.000000,0.000000,0,0.0,0
9,105,0.714,0.286,0,0,0,0,0,0,0.000,...,1,0,Formulation 10,1000.000000,0.000000,1.000000,0.000000,0,0.0,0


In [23]:
results_normalized = sdl.normalize(results_final)
results_normalized[['Formulation Quality','Size', 'Size_STD', 'Solu', 'Solu_STD', 'PDI', 'PDI_STD','Complexity']]

,Formulation Quality,Size,Size_STD,Solu,Solu_STD,PDI,PDI_STD,Complexity
0,1,0.236493,0.005762,0.559,0.0,0.191319,0.041543,0.166667
1,0,1.000000,0.000000,0.000,0.0,1.000000,0.000000,0.083333
2,1,0.230062,0.001216,0.559,0.0,0.320099,0.054492,0.166667
3,0,1.000000,0.000000,0.000,0.0,1.000000,0.000000,0.083333
4,1,0.233580,0.001771,0.556,0.0,0.282933,0.019797,0.250000
5,0,1.000000,0.000000,0.000,0.0,1.000000,0.000000,0.083333
6,0,1.000000,0.000000,0.000,0.0,1.000000,0.000000,0.083333
7,0,1.000000,0.000000,0.000,0.0,1.000000,0.000000,0.166667
8,0,1.000000,0.000000,0.000,0.0,1.000000,0.000000,0.083333
9,0,1.000000,0.000000,0.000,0.0,1.000000,0.000000,0.083333


In [24]:
results_normalized = sdl.normalize(results_final)
results_normalized[['Formulation Quality','Size', 'Size_STD', 'Solu', 'Solu_STD', 'PDI', 'PDI_STD','Complexity']]

,Formulation Quality,Size,Size_STD,Solu,Solu_STD,PDI,PDI_STD,Complexity
0,1,0.236493,0.005762,0.559,0.0,0.191319,0.041543,0.166667
1,0,1.000000,0.000000,0.000,0.0,1.000000,0.000000,0.083333
2,1,0.230062,0.001216,0.559,0.0,0.320099,0.054492,0.166667
3,0,1.000000,0.000000,0.000,0.0,1.000000,0.000000,0.083333
4,1,0.233580,0.001771,0.556,0.0,0.282933,0.019797,0.250000
5,0,1.000000,0.000000,0.000,0.0,1.000000,0.000000,0.083333
6,0,1.000000,0.000000,0.000,0.0,1.000000,0.000000,0.083333
7,0,1.000000,0.000000,0.000,0.0,1.000000,0.000000,0.166667
8,0,1.000000,0.000000,0.000,0.0,1.000000,0.000000,0.083333
9,0,1.000000,0.000000,0.000,0.0,1.000000,0.000000,0.083333


In [25]:
labeled_data_path = 'data/labeled_' + str(iteration) + '.xlsx'
labeled_norm_path = 'data/labeled_norm_' + str(iteration) + '.xlsx'


In [26]:
results_final.to_excel(labeled_data_path, index=False)
results_normalized.to_excel(labeled_norm_path, index=False)

# Load the labeled data to update the optimizer

In [27]:
labeled_norm = pd.read_excel(labeled_norm_path)
labeled_norm.head()

,trial_index,Drug,SL_1,SL_2,SL_3,LL_1,LL_2,LL_3,P_1,P_2,...,Complexity,Complexity_STD,Formulation,Size,Size_STD,PDI,PDI_STD,Formulation Quality,Solu,Solu_STD
0,96,0.559,0.441,0,0,0,0,0,0,0.000,...,0.166667,0,Formulation 1,0.236493,0.005762,0.191319,0.041543,1,0.559,0
1,97,0.714,0.286,0,0,0,0,0,0,0.000,...,0.083333,0,Formulation 2,1.000000,0.000000,1.000000,0.000000,0,0.000,0
2,98,0.559,0.441,0,0,0,0,0,0,0.000,...,0.166667,0,Formulation 3,0.230062,0.001216,0.320099,0.054492,1,0.559,0
3,99,0.714,0.286,0,0,0,0,0,0,0.000,...,0.083333,0,Formulation 4,1.000000,0.000000,1.000000,0.000000,0,0.000,0
4,100,0.556,0.438,0,0,0,0,0,0,0.006,...,0.250000,0,Formulation 5,0.233580,0.001771,0.282933,0.019797,1,0.556,0


In [28]:
ax_client_design = AxClient.load_from_json_file(ax_client_design_path)


In [29]:
loaded_optimizer_path = "optimizer/optimizer_load_" + str(iteration) + ".json"

In [30]:
loaded_ax_client = sdl.load_labeled_data(ax_client_design, labeled_norm_path)
loaded_ax_client.save_to_json_file(loaded_optimizer_path)

[INFO 08-07 06:35:21] ax.service.ax_client: Completed trial 96 with data: {'Solu': (0.559, 0), 'Size': (0.236493, 0.005762), 'PDI': (0.191319, 0.041543)}.
[INFO 08-07 06:35:21] ax.service.ax_client: Completed trial 97 with data: {'Solu': (0.0, 0), 'Size': (1.0, 0.0), 'PDI': (1.0, 0.0)}.
[INFO 08-07 06:35:21] ax.service.ax_client: Completed trial 98 with data: {'Solu': (0.559, 0), 'Size': (0.230062, 0.001216), 'PDI': (0.320099, 0.054492)}.
[INFO 08-07 06:35:21] ax.service.ax_client: Completed trial 99 with data: {'Solu': (0.0, 0), 'Size': (1.0, 0.0), 'PDI': (1.0, 0.0)}.
[INFO 08-07 06:35:21] ax.service.ax_client: Completed trial 100 with data: {'Solu': (0.556, 0), 'Size': (0.23358, 0.001771), 'PDI': (0.282933, 0.019797)}.
[INFO 08-07 06:35:21] ax.service.ax_client: Completed trial 101 with data: {'Solu': (0.0, 0), 'Size': (1.0, 0.0), 'PDI': (1.0, 0.0)}.
[INFO 08-07 06:35:21] ax.service.ax_client: Completed trial 102 with data: {'Solu': (0.0, 0), 'Size': (1.0, 0.0), 'PDI': (1.0, 0.0)}.
[